In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import random
import rtsvg
rt = rtsvg.RACETrack()

df = pl.read_csv('../../data/2013_vast_challenge/mc3_netflow/nf/nf-chunk1.csv')
df = rt.columnsAreTimestamps(df, 'parsedDate')
df = df.rename({'TimeSeconds':                '_del1_',
                'parsedDate':                 'timestamp',
                'dateTimeStr':                '_del2_',
                'ipLayerProtocol':            'pro',
                'ipLayerProtocolCode':        '_del3_',
                'firstSeenSrcIp':             'sip',
                'firstSeenDestIp':            'dip',
                'firstSeenSrcPort':           'spt',
                'firstSeenDestPort':          'dpt',
                'moreFragments':              '_del4_',
                'contFragments':              '_del5_',
                'durationSeconds':            'dur',
                'firstSeenSrcPayloadBytes':   '_del6_',
                'firstSeenDestPayloadBytes':  '_del7_',
                'firstSeenSrcTotalBytes':     'soct',
                'firstSeenDestTotalBytes':    'doct',
                'firstSeenSrcPacketCount':    'spkt',
                'firstSeenDestPacketCount':   'dpkt',
                'recordForceOut':             '_del8_'})
df = df.drop(['_del1_', '_del2_', '_del3_', '_del4_', '_del5_', '_del6_', '_del7_', '_del8_'])
#df = df.sample(100_000)

In [ ]:
#
# spreadLines() - attempt to implement this visualization
#
# Based on:
#
# @misc{kuo2024spreadlinevisualizingegocentricdynamic,
#       title={SpreadLine: Visualizing Egocentric Dynamic Influence}, 
#       author={Yun-Hsin Kuo and Dongyu Liu and Kwan-Liu Ma},
#       year={2024},
#       eprint={2408.08992},
#       archivePrefix={arXiv},
#       primaryClass={cs.HC},
#       url={https://arxiv.org/abs/2408.08992}, 
# }
# 
def spreadLines(rt_self,
                df,
                relationships,
                node_focus,
                ts_field        = None,  # Will attempt to guess based on datatypes
                every           = '1d',  # "the every field for the group_by_dynamic" ... 1d, 1h, 1m
                color_by        = None,
                count_by        = None,
                count_by_set    = False,
                widget_id       = None,
                w               = 1024,
                h               = 512,
                x_ins           = 32,
                y_ins           = 8,
                txt_h           = 12):
    if rt_self.isPolars(df) == False: raise Exception('spreadLines() - only supports polars dataframe')
    return SpreadLines(**locals())

#
#
#
class SpreadLines(object):
    #
    # transform all fields (if they area t-field)
    # - replace those fields w/ the new versions (i actually don't think the names change...)
    #
    def __transformFields__(self):
        # Gather up all of the fields that are going to be used
        _all_columns_ = [self.ts_field]
        if self.color_by is not None: _all_columns_.append(self.color_by)
        if self.count_by is not None: _all_columns_.append(self.count_by)
        for _relationship_ in self.relationships:
            _fm_, _to_ = _relationship_[0], _relationship_[1]
            if   type(_fm_) is str: _all_columns_.append(_fm_)
            elif type(_fm_) is tuple:
                for i in range(len(_fm_)): _all_columns_.append(_fm_[i])
            if   type(_to_) is str: _all_columns_.append(_to_)
            elif type(_to_) is tuple:
                for i in range(len(_to_)): _all_columns_.append(_to_[i])
        # Transform the fields
        self.df, _new_columns_ = self.rt_self.transformFieldListAndDataFrame(self.df, _all_columns_)
        # Remap them
        col_i = 0
        self.ts_field        = _new_columns_[col_i]
        col_i += 1
        if self.color_by is not None: 
            self.color_by = _new_columns_[col_i]
            col_i += 1
        if self.count_by is not None:
            self.count_by = _new_columns_[col_i]
            col_i += 1
        _new_relationships_ = []
        for _relationship_ in self.relationships:
            _fm_, _to_ = _relationship_[0], _relationship_[1]
            if   type(_fm_) is str: 
                _fm_ = _new_columns_[col_i]
                col_i += 1
            elif type(_fm_) is tuple:
                as_list = []
                for i in range(len(_fm_)):
                    as_list.append(_new_columns_[col_i])                    
                    col_i += 1
                _fm_ = tuple(as_list)
            if   type(_to_) is str: 
                _to_ = _new_columns_[col_i]
                col_i += 1
            elif type(_to_) is tuple:
                as_list = []
                for i in range(len(_to_)): 
                    as_list.append(_new_columns_[col_i])
                    col_i += 1
                _to_ = tuple(as_list)
            _new_relationships_.append((_fm_, _to_))
        self.relationships = _new_relationships_


    #
    # __consolidateRelationships__() - simplify the relationship fields into a single field
    # ... and use standard naming
    # ... replaces the "relationships" field w/ the consolidated field names
    # ... use (__fm0__, __to0__),( __fm1__, __to1__), etc.
    #
    def __consolidateRelationships__(self):
        new_relationships = []
        for i in range(len(self.relationships)):
            _fm_, _to_ = self.relationships[i]
            new_fm = f'__fm{i}__'
            new_to = f'__to{i}__'
            if type(_fm_) is str: self.df = self.df.with_columns(pl.col(_fm_).alias(new_fm))
            else:                 self.df = self.rt_self.createConcatColumn(self.df, _fm_, new_fm)
            if type(_to_) is str: self.df = self.df.with_columns(pl.col(_to_).alias(new_to))
            else:                 self.df = self.rt_self.createConcatColumn(self.df, _to_, new_to)
            new_relationships.append((new_fm, new_to))
        self.relationships = new_relationships

    #
    #
    #
    def __init__(self, rt_self, **kwargs):
        self.rt_self       = rt_self
        self.df            = rt_self.copyDataFrame(kwargs['df'])
        self.relationships = kwargs['relationships']
        self.node_focus    = kwargs['node_focus']
        self.ts_field      = self.rt_self.guessTimestampField(self.df) if kwargs['ts_field'] is None else kwargs['ts_field']
        self.every         = kwargs['every']
        self.color_by      = kwargs['color_by']
        self.count_by      = kwargs['count_by']
        self.count_by_set  = kwargs['count_by_set']
        self.widget_id     = f'spreadlines_{random.randint(0,65535)}' if kwargs['widget_id'] is None else kwargs['widget_id']
        self.w             = kwargs['w']
        self.h             = kwargs['h']
        self.x_ins         = kwargs['x_ins']
        self.y_ins         = kwargs['y_ins']
        self.txt_h         = kwargs['txt_h']
        # Unwrap any fields w/ the appropriate transforms
        self.__transformFields__()
        # Consolidate the fm's and to's into a simple field (__fm0__, __to0__),( __fm1__, __to1__), etc.
        self.__consolidateRelationships__()
        # How many bins?  And what's in those bins for nodes next to the focus?
        self.df = self.df.sort(self.ts_field)
        _bin_                    = 0
        _dfs_containing_focus_   = [] # focus  -> alter1 or alter1 -> focus
        _dfs_containing_alter2s_ = [] # alter1 -> alter2 or alter2 -> alter1  ... note does not include focus or alter1 <-> alter1
        self.bin_to_timestamps   = {}
        self.bin_to_alter1s      = {}
        self.bin_to_alter2s      = {}
        for k, k_df in self.df.group_by_dynamic(self.ts_field, every=self.every):
            _timestamp_     = k[0]
            _found_matches_ = False
            # find the first alters
            for i in range(len(self.relationships)):
                _fm_, _to_ = self.relationships[i]
                
                # From Is Focus
                _df_fm_is_focus_ = k_df.filter(pl.col(_fm_) == self.node_focus)
                _df_fm_is_focus_ = _df_fm_is_focus_.with_columns(pl.lit(_fm_).alias('__focus_col__'), pl.lit(_to_).alias('__alter_col__'), pl.lit(1).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('to').alias('__alter_side__'))
                if len(_df_fm_is_focus_) > 0: 
                    _dfs_containing_focus_.append(_df_fm_is_focus_)
                    if _bin_ not in self.bin_to_alter1s:        self.bin_to_alter1s[_bin_]       = {}
                    if 'to'  not in self.bin_to_alter1s[_bin_]: self.bin_to_alter1s[_bin_]['to'] = set()
                    self.bin_to_alter1s[_bin_]['to'] |= set(_df_fm_is_focus_[_to_])
                    _found_matches_ = True

                # To Is Focus
                _df_to_is_focus_ = k_df.filter(pl.col(_to_) == self.node_focus)
                _df_to_is_focus_ = _df_to_is_focus_.with_columns(pl.lit(_to_).alias('__focus_col__'), pl.lit(_fm_).alias('__alter_col__'), pl.lit(1).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('fm').alias('__alter_side__'))
                if len(_df_to_is_focus_) > 0:
                    _dfs_containing_focus_.append(_df_to_is_focus_)
                    if _bin_ not in self.bin_to_alter1s:        self.bin_to_alter1s[_bin_]       = {}
                    if 'fm'  not in self.bin_to_alter1s[_bin_]: self.bin_to_alter1s[_bin_]['fm'] = set()
                    self.bin_to_alter1s[_bin_]['fm'] |= set(_df_to_is_focus_[_fm_])
                    _found_matches_ = True

                # For any shared nodes between the two sides, keep them on the 'fm' side
                if _bin_ in self.bin_to_alter1s and 'fm' in self.bin_to_alter1s[_bin_] and 'to' in self.bin_to_alter1s[_bin_]:
                    _shared_nodes_ = self.bin_to_alter1s[_bin_]['fm'] & self.bin_to_alter1s[_bin_]['to']
                    if len(_shared_nodes_) > 0: self.bin_to_alter1s[_bin_]['to'] -= _shared_nodes_

            # find the second alters
            if _found_matches_:
                _all_alter1s_ = set()
                if 'fm' in self.bin_to_alter1s[_bin_]: _all_alter1s_ |= self.bin_to_alter1s[_bin_]['fm']
                if 'to' in self.bin_to_alter1s[_bin_]: _all_alter1s_ |= self.bin_to_alter1s[_bin_]['to']
                # Go through all the relationships
                for i in range(len(self.relationships)):
                    _fm_, _to_ = self.relationships[i]
                    if 'fm' in self.bin_to_alter1s[_bin_]:
                        _df_          = k_df.filter(pl.col(_fm_).is_in(self.bin_to_alter1s[_bin_]['fm']) | pl.col(_to_).is_in(self.bin_to_alter1s[_bin_]['fm']))
                        _set_alter2s_ = (set(_df_[_fm_]) | set(_df_[_to_])) - (_all_alter1s_ | set([self.node_focus]))
                        if len(_set_alter2s_) > 0:
                            if _bin_ not in self.bin_to_alter2s:        self.bin_to_alter2s[_bin_]       = {}
                            if 'fm'  not in self.bin_to_alter2s[_bin_]: self.bin_to_alter2s[_bin_]['fm'] = set()
                            self.bin_to_alter2s[_bin_]['fm'] |= _set_alter2s_

                            _df_ = k_df.filter(pl.col(_fm_).is_in(self.bin_to_alter1s[_bin_]['fm']) & pl.col(_to_).is_in(_set_alter2s_))
                            _df_ = _df_.with_columns(pl.lit(_fm_).alias('__alter1_col__'), pl.lit(_to_).alias('__alter2_col__'), pl.lit(2).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('fm').alias('__alter_side__'))
                            _dfs_containing_alter2s_.append(_df_)

                            _df_ = k_df.filter(pl.col(_to_).is_in(self.bin_to_alter1s[_bin_]['fm']) & pl.col(_fm_).is_in(_set_alter2s_))
                            _df_ = _df_.with_columns(pl.lit(_to_).alias('__alter1_col__'), pl.lit(_fm_).alias('__alter2_col__'), pl.lit(2).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('fm').alias('__alter_side__'))
                            _dfs_containing_alter2s_.append(_df_)

                    if 'to' in self.bin_to_alter1s[_bin_]:
                        _df_          = k_df.filter(pl.col(_fm_).is_in(self.bin_to_alter1s[_bin_]['to']) | pl.col(_to_).is_in(self.bin_to_alter1s[_bin_]['to']))
                        _set_alter2s_ = (set(_df_[_fm_]) | set(_df_[_to_])) - (_all_alter1s_ | set([self.node_focus]))
                        if len(_set_alter2s_) > 0:
                            if _bin_ not in self.bin_to_alter2s:        self.bin_to_alter2s[_bin_]       = {}
                            if 'to'  not in self.bin_to_alter2s[_bin_]: self.bin_to_alter2s[_bin_]['to'] = set()
                            self.bin_to_alter2s[_bin_]['to'] |= _set_alter2s_

                            _df_ = k_df.filter(pl.col(_fm_).is_in(self.bin_to_alter1s[_bin_]['to']) & pl.col(_to_).is_in(_set_alter2s_))
                            _df_ = _df_.with_columns(pl.lit(_fm_).alias('__alter1_col__'), pl.lit(_to_).alias('__alter2_col__'), pl.lit(2).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('to').alias('__alter_side__'))
                            _dfs_containing_alter2s_.append(_df_)

                            _df_ = k_df.filter(pl.col(_to_).is_in(self.bin_to_alter1s[_bin_]['to']) & pl.col(_fm_).is_in(_set_alter2s_))
                            _df_ = _df_.with_columns(pl.lit(_to_).alias('__alter1_col__'), pl.lit(_fm_).alias('__alter2_col__'), pl.lit(2).alias('__alter_level__'), pl.lit(_bin_).alias('__bin__'), pl.lit(_timestamp_).alias('__bin_ts__'), pl.lit('to').alias('__alter_side__'))
                            _dfs_containing_alter2s_.append(_df_)

                # For any shared nodes between the two sides, keep them on the 'fm' side
                if _bin_ in self.bin_to_alter2s and 'fm' in self.bin_to_alter2s[_bin_] and 'to' in self.bin_to_alter2s[_bin_]:
                    _shared_nodes_ = self.bin_to_alter2s[_bin_]['fm'] & self.bin_to_alter2s[_bin_]['to']
                    if len(_shared_nodes_) > 0: self.bin_to_alter2s[_bin_]['to'] -= _shared_nodes_

            if _found_matches_: 
                self.bin_to_timestamps[_bin_] = _timestamp_
                _bin_ += 1
        
        # Concatenate the pieces and parts
        if len(_dfs_containing_focus_) > 0:   self.df_alter1s = pl.concat(_dfs_containing_focus_).unique()    # unique because we may have duplicate rows on the two sides
        else:                                 self.df_alter1s = pl.DataFrame()
        if len(_dfs_containing_alter2s_) > 0: self.df_alter2s = pl.concat(_dfs_containing_alter2s_).unique()  # unique because we may have duplicate rows on the two sides
        else:                                 self.df_alter2s = pl.DataFrame()

    # nodesInBin() - return the set of nodes that exist in this bin
    def nodesInBin(self, bin):
        nodes_in_this_bin = set()
        if bin in self.bin_to_alter1s and 'fm' in self.bin_to_alter1s[bin]: nodes_in_this_bin |= self.bin_to_alter1s[bin]['fm']
        if bin in self.bin_to_alter1s and 'to' in self.bin_to_alter1s[bin]: nodes_in_this_bin |= self.bin_to_alter1s[bin]['to']
        if bin in self.bin_to_alter2s and 'fm' in self.bin_to_alter2s[bin]: nodes_in_this_bin |= self.bin_to_alter2s[bin]['fm']
        if bin in self.bin_to_alter2s and 'to' in self.bin_to_alter2s[bin]: nodes_in_this_bin |= self.bin_to_alter2s[bin]['to']
        return nodes_in_this_bin

    # nodesExistInOtherBins() - return the set of nodes that exist in this bin AND'ed with all the other bins
    def nodesExistsInOtherBins(self, bin):
        nodes_in_this_bin = self.nodesInBin(bin)
        all_other_bins    = set()
        for _bin_ in (self.bin_to_alter1s.keys()|self.bin_to_alter2s.keys()):
            if _bin_ == bin: continue
            all_other_bins |= self.nodesInBin( _bin_)
        return nodes_in_this_bin & all_other_bins

    #
    # svgSketch() - produce a basic sketch of how many nodes would occur where in the final rendering...
    #
    def svgSketch(self):
        w_usable, h_usable = self.w - 2*self.x_ins, self.h - 2*self.y_ins
        y_mid              = self.y_ins + h_usable/2
        bin_to_x           = {}
        bin_inter_dist     = w_usable/(len(self.bin_to_alter1s) - 1)
        for _bin_ in self.bin_to_alter1s: bin_to_x[_bin_] = self.x_ins + _bin_*bin_inter_dist
        _y_diff_alter1s_, _y_diff_alter2s_ = h_usable/8, 2*h_usable/8

        svg = [f'<svg x="0" y="0" width="{self.w}" height="{self.h}">']
        svg.append(f'<rect x="0" y="0" width="{self.w}" height="{self.h}" fill="{self.rt_self.co_mgr.getTVColor("background","default")}" />')

        svg.append(f'<line x1="{self.x_ins}" y1="{y_mid}" x2="{self.x_ins+w_usable}" y2="{y_mid}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="4" />')        
        for _bin_ in bin_to_x:
            _x_ = bin_to_x[_bin_]
            svg.append(f'<line x1="{_x_}" y1="{self.y_ins}" x2="{_x_}" y2="{self.y_ins + h_usable}" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" />')
            svg.append(f'<circle cx="{_x_}" cy="{y_mid}" r="5" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" fill="{self.rt_self.co_mgr.getTVColor('data','default')}" />')
            _date_str_ = self.bin_to_timestamps[_bin_].strftime(self.__dateFormat__())
            svg.append(self.rt_self.svgText(_date_str_, _x_-2, self.y_ins + h_usable + 4, rt.co_mgr.getTVColor('axis','minor'), anchor='begin', rotation=270))
            if _bin_ in self.bin_to_alter1s and 'fm' in self.bin_to_alter1s[_bin_]: # top of the image
                _y_         = y_mid - _y_diff_alter1s_
                _num_nodes_ = len(self.bin_to_alter1s[_bin_]['fm'])
                svg.append(self.rt_self.svgText(str(_num_nodes_), _x_+2, _y_ + 4, 'black', anchor='begin', rotation=90))
                if _bin_ in self.bin_to_alter2s and 'fm' in self.bin_to_alter2s[_bin_]:
                    _y_         = y_mid - _y_diff_alter2s_
                    _num_nodes_ = len(self.bin_to_alter2s[_bin_]['fm'])
                    svg.append(self.rt_self.svgText(str(_num_nodes_), _x_+2, _y_ + 4, 'black', anchor='begin', rotation=90))
            if _bin_ in self.bin_to_alter1s and 'to' in self.bin_to_alter1s[_bin_]: # bottom of the image
                _y_         = y_mid + _y_diff_alter1s_
                _num_nodes_ = len(self.bin_to_alter1s[_bin_]['to'])
                svg.append(self.rt_self.svgText(str(_num_nodes_), _x_+2, _y_ + 4, 'black', anchor='begin', rotation=90))
                if _bin_ in self.bin_to_alter2s and 'to' in self.bin_to_alter2s[_bin_]:
                    _y_         = y_mid + _y_diff_alter2s_
                    _num_nodes_ = len(self.bin_to_alter2s[_bin_]['to'])
                    svg.append(self.rt_self.svgText(str(_num_nodes_), _x_+2, _y_ + 4, 'black', anchor='begin', rotation=90))

        svg.append('</svg>')
        return ''.join(svg)

    def __dateFormat__(self):
        if   'd' in self.every: return '%Y-%m-%d'
        elif 'h' in self.every: return '%Y-%m-%d %H'
        else:                   return '%Y-%m-%d %H:%M'

    def packable(self, nodes, x, y, y_max, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer):
        node_to_xy = {}
        h = abs(y - y_max)
        n = len(nodes)
        if n > 0:
            # single strand
            r = ((h - (n-1)*circle_inter_d)/n)/2.0
            if r >= r_min:
                r          = min(r, r_pref)
                left_overs = 0
                out_of     = n
                for _node_i_ in range(len(nodes)):
                    _node_ = nodes[-(_node_i_+1)]
                    #if mul == -1: _node_ = nodes[_node_i_]
                    #else:         _node_ = nodes[-(_node_i_+1)]
                    node_to_xy[_node_] = (x, y+mul*r, r)
                    y += mul*(2*r+circle_inter_d)
            else:
                # m-strands
                m_max = w_max / (2*r_min+circle_spacer)
                for m in range(2,int(m_max)+1):
                    r = (h - (n//m)*circle_inter_d)/(n//m)/2.0
                    if r >= r_min:
                        r = min(r, r_pref)
                        total_width_required = m*(2*r) + (m-1)*circle_spacer
                        if total_width_required > w_max: continue
                        _col_, nodes_in_this_column = 0, 0
                        nodes_per_column = n//m
                        left_overs       = n - nodes_per_column*m
                        out_of           = nodes_per_column
                        if left_overs > 0: m += 1
                        total_width_required = m*(2*r) + (m-1)*circle_spacer
                        _columns_ = []
                        _column_  = []
                        for _node_ in nodes:
                            _x_col_ = x - total_width_required/2.0 + _col_*(2*r+circle_spacer) + r
                            _y_row_ = y+mul*r+mul*nodes_in_this_column*(2*r+circle_inter_d)                        
                            _column_.append((_x_col_, _y_row_))
                            nodes_in_this_column += 1
                            if nodes_in_this_column >= nodes_per_column: 
                                _columns_.append(_column_)
                                _column_  = []
                                _col_, nodes_in_this_column = _col_+1, 0
                        if len(_column_) > 0: _columns_.append(_column_)
                        # Allocate the across first... and then down...
                        _xi_, _yi_ = 0, 0
                        for _node_i_ in range(len(nodes)):
                            if mul == -1: _node_ = nodes[len(nodes) - 1 - _node_i_]
                            else:         _node_ = nodes[_node_i_]
                            if _yi_ >= len(_columns_[_xi_]): _yi_, _xi_ = _yi_ + 1, 0
                            _xy_ = _columns_[_xi_][_yi_]
                            node_to_xy[_node_] = (_xy_[0], _xy_[1], r) 
                            _xi_ += 1
                            if _xi_ >= len(_columns_): _yi_, _xi_ = _yi_ + 1, 0
                        break

        if len(node_to_xy) == 0: return None, None, None
        return node_to_xy, left_overs, out_of

    def renderAlter(self, nodes, befores, afters, x, y, y_max, w_max, mul=1, r_min=4.0, r_pref=7.0, circle_inter_d=2.0, circle_spacer=3, h_collapsed_sections=16):
        xmin, ymin, xmax, ymax = x-r_pref-circle_inter_d, y-r_pref-circle_inter_d, x+r_pref+circle_inter_d, y+r_pref+circle_inter_d
        # Create the started/stopped triangles for a single node
        def svgTriangle(x,y,r,s,d):
            nonlocal xmin, ymin, xmax, ymax
            p0      = (x+d*(r/2.0), y)
            p1      = (x+d*(r+s),   y+r)
            p2      = (x+d*(r+s),   y-r)
            for _pt_ in [p0,p1,p2]: xmin, ymin, xmax, ymax = min(xmin, _pt_[0]), min(ymin, _pt_[1]), max(xmax, _pt_[0]), max(ymax, _pt_[1])
            _path_  = f'M {p0[0]} {p0[1]} L {p1[0]} {p1[1]} L {p2[0]} {p2[1]} Z'
            _color_ = '#ff0000' if d == 1 else '#0000ff'
            return f'<path d="{_path_}" stroke="none" fill="{_color_}" />'
        # Create the started/stopped triangles for the clouds
        def svgCloudTriangle(x,y,offset,s,d):
            nonlocal xmin, ymin, xmax, ymax
            p0      = (x+d*(offset), y)
            p1      = (x+d*(offset+s),   y+s)
            p2      = (x+d*(offset+s),   y-s)
            for _pt_ in [p0,p1,p2]: xmin, ymin, xmax, ymax = min(xmin, _pt_[0]), min(ymin, _pt_[1]), max(xmax, _pt_[0]), max(ymax, _pt_[1])
            _path_  = f'M {p0[0]} {p0[1]} L {p1[0]} {p1[1]} L {p2[0]} {p2[1]} Z'
            _color_ = '#d3494e' if d == 1 else '#658cbb'
            return f'<path d="{_path_}" stroke="none" fill="{_color_}" />'
        # Place the nodes onto the canvas
        def placeNodeToXYs(n2xy):
            nonlocal xmin, ymin, xmax, ymax
            for _node_, _xyr_ in n2xy.items():
                svg.append(f'<circle cx="{_xyr_[0]}" cy="{_xyr_[1]}" r="{_xyr_[2]}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.75" fill="none"/>')
                xmin, ymin, xmax, ymax = min(xmin, _xyr_[0]-_xyr_[2]), min(ymin, _xyr_[1]-_xyr_[2]), max(xmax, _xyr_[0]+_xyr_[2]), max(ymax, _xyr_[1]+_xyr_[2])
                if _node_ not in befores: svg.append(svgTriangle(_xyr_[0], _xyr_[1], _xyr_[2], circle_spacer/2, -1))
                if _node_ not in afters:  svg.append(svgTriangle(_xyr_[0], _xyr_[1], _xyr_[2], circle_spacer/2,  1))
        # Render the summarization cloud
        def summarizationCloud(n, y_cloud, ltriangle, rtriangle):
            nonlocal xmin, ymin, xmax, ymax
            svg.append(self.rt_self.iconCloud(x,y_cloud, fg='#e0e0e0', bg='#e0e0e0'))
            if ltriangle: svg.append(svgCloudTriangle(x, y_cloud, 16, 6, -1))
            if rtriangle: svg.append(svgCloudTriangle(x, y_cloud, 16, 6,  1))
            svg.append(self.rt_self.svgText(str(n), x, y_cloud + 4, 'black', anchor='middle'))
            xmin, ymin, xmax, ymax = min(xmin, x-16), min(ymin, y_cloud-6), max(xmax, x+16), max(ymax, y_cloud+6)
        # Render the main SVG ... geometry and some guide lines (for reference/debug, commented out now)
        h       = abs(y_max - y)
        svg     = []
        #svg.append(f'<line x1="{x-w_max/2.0}" y1="{y}"     x2="{x+w_max/2.0}" y2="{y}"     stroke="#0000ff" stroke-width="4.0" />') # render the "start"
        #svg.append(f'<line x1="{x-w_max/2.0}" y1="{y_max}" x2="{x+w_max/2.0}" y2="{y_max}" stroke="#ff0000" stroke-width="0.8" />')
        #svg.append(f'<line x1="{x}"           y1="{0}"     x2="{x}"           y2="{384}"   stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" />')
        # Make sure there are nodes...
        if len(nodes) > 0:
            # Sort the nodes into the 4 categories
            nodes_sorter = []
            nodes_isolated, nodes_started, nodes_stopped, nodes_continuous = [], [], [], []
            for _node_ in nodes:
                if   _node_ in befores and _node_ in afters: nodes_sorter.append((3, _node_)), nodes_continuous.append(_node_)
                elif _node_ in befores:                      nodes_sorter.append((2, _node_)), nodes_stopped   .append(_node_)
                elif _node_ in afters:                       nodes_sorter.append((1, _node_)), nodes_started   .append(_node_)
                else:                                        nodes_sorter.append((0, _node_)), nodes_isolated  .append(_node_)
            nodes_sorter  = sorted(nodes_sorter)
            nodes_ordered = [x[1] for x in nodes_sorter]

            # Try putting them all down first... which won't work for any non-trivial number of nodes
            node_to_xy, leftovers, out_of = self.packable(nodes_ordered, x, y, y_max, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer)
            if node_to_xy is not None:
                placeNodeToXYs(node_to_xy) # no summarization necessary
            else:
                top_adjust = h_collapsed_sections if mul == 1 else -h_collapsed_sections
                node_to_xy, leftovers, out_of = self.packable(nodes_started+nodes_stopped+nodes_continuous, x, y, y_max-top_adjust, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer)
                if node_to_xy is not None:
                    placeNodeToXYs(node_to_xy) # summarize isolated nodes only
                    y_off = ymin if mul == 1 else ymax
                    summarizationCloud(len(nodes_isolated), y_off+mul*0.5*h_collapsed_sections, True, True)
                else:
                    top_adjust = 2*h_collapsed_sections if mul == 1 else -2*h_collapsed_sections
                    node_to_xy, leftovers, out_of = self.packable(nodes_started              +nodes_continuous, x, y, y_max-top_adjust, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer)
                    if node_to_xy is not None:
                        placeNodeToXYs(node_to_xy) # summarize isolated nodes and nodes_stopped
                        y_off = ymax if mul == 1 else ymin
                        summarizationCloud(len(nodes_stopped),  y_off+mul*0.5*h_collapsed_sections, False,  True)
                        summarizationCloud(len(nodes_isolated), y_off+mul*1.5*h_collapsed_sections, True,   True)
                    else:
                        node_to_xy, leftovers, out_of = self.packable(nodes_stopped+nodes_continuous, x, y, y_max-top_adjust, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer)
                        if node_to_xy is not None:
                            placeNodeToXYs(node_to_xy) # summarize isolated nodes and nodes_started
                            y_off = ymax if mul == 1 else ymin
                            summarizationCloud(len(nodes_started),   y_off+mul*0.5*h_collapsed_sections, True,  False)
                            summarizationCloud(len(nodes_isolated),  y_off+mul*1.5*h_collapsed_sections, True,  True)
                        else:
                            top_adjust = 3*h_collapsed_sections if mul == 1 else -3*h_collapsed_sections
                            node_to_xy, leftovers, out_of = self.packable(nodes_continuous, x, y, y_max-top_adjust, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer)
                            if node_to_xy is not None:
                                placeNodeToXYs(node_to_xy) # summarize everyting but the continuous nodes (nodes seen in both directions)
                                y_off = ymax if mul == 1 else ymin
                                summarizationCloud(len(nodes_started),   y_off+mul*0.5*h_collapsed_sections, True,  False)
                                summarizationCloud(len(nodes_stopped),   y_off+mul*1.5*h_collapsed_sections, False, True)
                                summarizationCloud(len(nodes_isolated),  y_off+mul*2.5*h_collapsed_sections, True,  True)
                            else:
                                # everything is summarized :(
                                summarizationCloud(len(nodes_continuous), y+mul*0.5*h_collapsed_sections, False,  False)
                                summarizationCloud(len(nodes_started),    y+mul*1.5*h_collapsed_sections, True,   False)
                                summarizationCloud(len(nodes_stopped),    y+mul*2.5*h_collapsed_sections, False,  True)
                                summarizationCloud(len(nodes_isolated),   y+mul*3.5*h_collapsed_sections, True,   True)
        
        xmin, ymin, xmax, ymax = xmin - r_pref, ymin - r_pref, xmax + r_pref, ymax + r_pref
        # svg.append(f'<rect x="{xmin}" y="{ymin}" width="{xmax-xmin}" height="{ymax-ymin}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
        return ''.join(svg), (xmin, ymin, xmax, ymax)

#
# spreadLines()
#
sl = spreadLines(rt, df, [('sip','dip')], '172.30.0.4', every="4h", h=384)
rt.tile([sl.svgSketch()])

In [ ]:
#rt.histogram(df, bin_by='sip', count_by='dip')

In [ ]:
#sl = spreadLines(rt, df, [('sip','dip')], '172.10.0.6', every="1h", h=384) # highest out-degree node
#rt.tile([sl.svgSketch()])

In [ ]:
#rt.histogram(df, bin_by='dip', count_by='sip')

In [ ]:
sl = spreadLines(rt, df, [('sip','dip')], '172.0.0.1', every="1h", h=384) # highest in-degree node
rt.tile([sl.svgSketch()])

In [ ]:

def renderBin(self, bin, w_bin_min=16, h_bin=384, r_min=3.0, r_pref=5.0):
    svgTriangle = lambda x: f'<path d="M {x[0][0]} {x[0][1]} L {x[1][0]} {x[1][1]} L {x[2][0]} {x[2][1]} Z" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" fill="{self.rt_self.co_mgr.getTVColor("data","default")}" />'
    nodes_in_bin             = self.nodesInBin(bin)
    nodes_also_in_other_bins = self.nodesExistsInOtherBins(bin)

    svg = [f'<circle cx="0" cy="0" r="{r_pref}" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" fill="{self.rt_self.co_mgr.getTVColor("data","default")}" />']

    alter1_h_alloc = h_bin/6
    alter2_h_alloc = h_bin/6

    y = 0 - 2*r_pref 
    if 'fm' in self.bin_to_alter1s[bin]:
        if len(self.bin_to_alter1s[bin]['fm']) * r_min < alter1_h_alloc:
            r = alter1_h_alloc/len(self.bin_to_alter1s[bin]['fm'])/2
            if r < r_min:  r = r_min
            if r > r_pref: r = r_pref
            for _node_ in self.bin_to_alter1s[bin]['fm']:
                svg.append(f'<circle cx="0" cy="{y}" r="{r}" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" fill="none" />')
                if _node_ not in nodes_also_in_other_bins: svg.append(svgTriangle([( r,y), ( 2*r,y-r), ( 2*r,y+r)])), svg.append(svgTriangle([(-r,y), (-2*r,y+r), (-2*r,y-r)]))
                y -= 2*r

    y = 0 + 2*r_pref
    if 'to' in self.bin_to_alter1s[bin]:
        if len(self.bin_to_alter1s[bin]['to']) * r_min < alter1_h_alloc:
            r = alter1_h_alloc/len(self.bin_to_alter1s[bin]['fm'])/2
            if r < r_min:  r = r_min
            if r > r_pref: r = r_pref
            for _node_ in self.bin_to_alter1s[bin]['to']:
                svg.append(f'<circle cx="0" cy="{y}" r="{r}" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="1.0" fill="none" />')
                if _node_ not in nodes_also_in_other_bins: svg.append(svgTriangle([(r,y), (2*r, y-r), (2*r, y+r)])), svg.append(svgTriangle([(-r,y),(-2*r,y+r), (-2*r,y-r)]))
                y += 2*r

    return ''.join(svg), w_bin_min

_svg_, _bin_w_ = renderBin(sl, 2)
_hdr_ = f'<svg x="0" y="0" width="{384}" height="{384}" viewBox="-{384/2} -{384/2} {384} {384}">'
_bg_  = f'<rect x="{-384/2}" y="{-384/2}" width="{384}" height="{384}" fill="{rt.co_mgr.getTVColor("background","default")}" />'
_ftr_ = '</svg>'
#rt.tile([_hdr_ + _bg_ + _svg_ + _ftr_])

In [ ]:
# packable... works... but does the nodes in the wrong orientation...
def WORKS_packable(self, nodes, x, y, y_max, w_max, mul, r_min, r_pref, circle_inter_d, circle_spacer):
    node_to_xy = {}
    h = abs(y - y_max)
    n = len(nodes)
    if n > 0:
        # single strand
        r = ((h - (n-1)*circle_inter_d)/n)/2.0
        if r >= r_min:
            r          = min(r, r_pref)
            left_overs = 0
            out_of     = n
            for _node_ in nodes:
                node_to_xy[_node_] = (x, y+mul*r, r)
                y += mul*(2*r+circle_inter_d)
        else:
            # m-strands
            m_max = w_max / (2*r_min+circle_spacer)
            for m in range(2,int(m_max)+1):
                r = (h - (n//m)*circle_inter_d)/(n//m)/2.0
                if r >= r_min:
                    r = min(r, r_pref)
                    total_width_required = m*(2*r) + (m-1)*circle_spacer
                    if total_width_required > w_max: continue
                    _col_, nodes_in_this_column = 0, 0
                    nodes_per_column = n//m
                    left_overs       = n - nodes_per_column*m
                    out_of           = nodes_per_column
                    if left_overs > 0: m += 1
                    total_width_required = m*(2*r) + (m-1)*circle_spacer
                    for _node_ in nodes:
                        _x_col_ = x - total_width_required/2.0 + _col_*(2*r+circle_spacer) + r
                        _y_row_ = y+mul*r+mul*nodes_in_this_column*(2*r+circle_inter_d)
                        node_to_xy[_node_] = (_x_col_, _y_row_, r)
                        nodes_in_this_column += 1
                        if nodes_in_this_column >= nodes_per_column: _col_, nodes_in_this_column = _col_+1, 0
                    break
    if len(node_to_xy) == 0: return None, None, None
    return node_to_xy, left_overs, out_of


In [ ]:
# Demonstrates various configurations of a single alter render
_tiles_ = []
for _num_of_nodes_ in range(500, 1500, 300):
    _hdr_ = f'<svg x="0" y="0" width="{384}" height="{384}">'
    _bg_  = f'<rect x="0" y="0" width="{384}" height="{384}" fill="{rt.co_mgr.getTVColor("background","default")}" />'
    _ftr_ = '</svg>'
    _nodes_, _befores_, _afters_ = set(), set(), set()
    for i in range(_num_of_nodes_):
        _nodes_.add(i)
        if random.random() < 0.3: _befores_.add(i)
        if random.random() < 0.3: _afters_.add(i)
    _svg_, _bounds_ = sl.renderAlter(_nodes_, _befores_, _afters_, 175, 300, 200, 128, mul=-1)
    xmin, ymin, xmax, ymax = _bounds_
    _box_ = f'<rect x="{xmin}" y="{ymin}" width="{xmax-xmin}" height="{ymax-ymin}" stroke="{rt.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="10" />'
    _tiles_.append(_hdr_ + _bg_ + _svg_ + _box_ + _ftr_)
    _svg_, _bounds_ = sl.renderAlter(_nodes_, _befores_, _afters_, 175, 100, 300,  64, mul= 1)
    xmin, ymin, xmax, ymax = _bounds_
    _box_ = f'<rect x="{xmin}" y="{ymin}" width="{xmax-xmin}" height="{ymax-ymin}" stroke="{rt.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="10" />'
    _tiles_.append(_hdr_ + _bg_ + _svg_ + _box_ + _ftr_)
#rt.table(_tiles_, per_row=4, spacer=10)

In [ ]:
def renderBin(self, 
              bin,                        # bin index
              x,                          # center of the bin 
              y,                          # center of the bin
              max_w,                      # max width of the bin (i.e., the max width of any of the alters)
              max_h,                      # max height of the bin (halfed in each direction from y)
              r_min                = 4.0, 
              r_pref               = 7.0, 
              circle_inter_d       = 2.0, 
              circle_spacer        = 3,
              alter_separation_h   = 48, 
              h_collapsed_sections = 16):
    _all_nodes_in_this_bin = self.nodesInBin(bin)
    _nodes_in_other_bins_  = self.nodesExistsInOtherBins(bin)
    _befores_, _afters_    = set(), set()
    for i in range(bin):                                       _befores_ |= self.nodesInBin(i)
    for i in range(bin+1, len(self.bin_to_timestamps.keys())): _afters_  |= self.nodesInBin(i)
    svg         = [f'<circle cx="{x}" cy="{y}" r="{r_pref}" stroke="{self.rt_self.co_mgr.getTVColor("axis","minor")}" stroke-width="0.4" fill="{self.rt_self.co_mgr.getTVColor("data","default")}" />']
    max_alter_h = max_h/5.0
    # Approximations of the alters
    #svg.append(f'<rect x="{x-max_w/2}" y="{y-r_pref-max_alter_h}"                      width="{max_w}" height="{max_alter_h}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
    #svg.append(f'<rect x="{x-max_w/2}" y="{y-r_pref-2*max_alter_h-alter_separation_h}" width="{max_w}" height="{max_alter_h}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
    #svg.append(f'<rect x="{x-max_w/2}" y="{y+r_pref}"                                  width="{max_w}" height="{max_alter_h}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
    #svg.append(f'<rect x="{x-max_w/2}" y="{y+r_pref+  max_alter_h+alter_separation_h}" width="{max_w}" height="{max_alter_h}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')

    # Actual alters
    if 'fm' in self.bin_to_alter1s[bin]:
        _svg_, _bounds_ = self.renderAlter(self.bin_to_alter1s[bin]['fm'], _befores_, _afters_, x, y-r_pref-2*circle_inter_d, y-r_pref-max_alter_h,                  max_w, -1, r_min, r_pref, circle_inter_d, circle_spacer, h_collapsed_sections)
        svg.append(_svg_)
        alter1s_fm_bounds = _bounds_
    else:
        alter1s_fm_bounds = None
        _bounds_          = (x-r_pref, y-r_pref-2*circle_inter_d-5, x+r_pref, y-r_pref-2*circle_inter_d)
    #svg.append(f'<rect x="{_bounds_[0]}" y="{_bounds_[1]}" width="{_bounds_[2]-_bounds_[0]}" height="{_bounds_[3]-_bounds_[1]}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')

    if 'fm' in self.bin_to_alter2s[bin]:
        _svg_, _bounds_ = self.renderAlter(self.bin_to_alter2s[bin]['fm'], _befores_, _afters_, x, _bounds_[1]-alter_separation_h, _bounds_[1]-alter_separation_h-max_alter_h, max_w, -1, r_min, r_pref, circle_inter_d, circle_spacer, h_collapsed_sections)
        svg.append(_svg_)
        alter2s_fm_bounds = _bounds_
        #svg.append(f'<rect x="{_bounds_[0]}" y="{_bounds_[1]}" width="{_bounds_[2]-_bounds_[0]}" height="{_bounds_[3]-_bounds_[1]}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
    else:
        alter2s_fm_bounds = None

    if 'to' in self.bin_to_alter1s[bin]:
        _svg_, _bounds_ = self.renderAlter(self.bin_to_alter1s[bin]['to'], _befores_, _afters_, x, y+r_pref+2*circle_inter_d, y+r_pref+2*circle_inter_d+max_alter_h, max_w,  1, r_min, r_pref, circle_inter_d, circle_spacer, h_collapsed_sections)
        svg.append(_svg_)
        alter1s_to_bounds = _bounds_
    else: 
        _bounds_ = (x-r_pref, y+r_pref+2*circle_inter_d, x+r_pref, y+r_pref+2*circle_inter_d+5)
        alter1s_to_bounds = None
    #svg.append(f'<rect x="{_bounds_[0]}" y="{_bounds_[1]}" width="{_bounds_[2]-_bounds_[0]}" height="{_bounds_[3]-_bounds_[1]}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')

    if 'to' in self.bin_to_alter2s[bin]:
        _svg_, _bounds_ = self.renderAlter(self.bin_to_alter2s[bin]['to'], _befores_, _afters_, x, _bounds_[3]+alter_separation_h, _bounds_[3]+alter_separation_h+max_alter_h, max_w, 1, r_min, r_pref, circle_inter_d, circle_spacer, h_collapsed_sections)
        svg.append(_svg_)
        alter2s_to_bounds = _bounds_
        # svg.append(f'<rect x="{_bounds_[0]}" y="{_bounds_[1]}" width="{_bounds_[2]-_bounds_[0]}" height="{_bounds_[3]-_bounds_[1]}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="0.8" fill="none" rx="{r_pref}" />')
    else:
        alter2s_to_bounds = None

    # Calculate the outline of the bin
    overall_w = 2*r_pref
    if alter1s_fm_bounds is not None: overall_w = max(overall_w, alter1s_fm_bounds[2]-alter1s_fm_bounds[0])
    if alter1s_to_bounds is not None: overall_w = max(overall_w, alter1s_to_bounds[2]-alter1s_to_bounds[0])
    if alter2s_fm_bounds is not None: overall_w = max(overall_w, alter2s_fm_bounds[2]-alter2s_fm_bounds[0])
    if alter2s_to_bounds is not None: overall_w = max(overall_w, alter2s_to_bounds[2]-alter2s_to_bounds[0])
    narrow_w = overall_w - 2*r_pref
    _amt_    = 2*r_pref
    d_array  = [f'M {x-overall_w/2.0} {y}']
    if alter1s_to_bounds is None:
        d_array.append(f'L {x-overall_w/2.0} {y+  _amt_}  C {x-overall_w/2.0} {y+2*_amt_} {x-overall_w/2.0} {y+2*_amt_} {x-narrow_w/2.0}  {y+2*_amt_}')
        d_array.append(f'L {x+narrow_w/2.0}  {y+2*_amt_}  C {x+overall_w/2.0} {y+2*_amt_} {x+overall_w/2.0} {y+2*_amt_} {x+overall_w/2.0} {y+  _amt_}')
        d_array.append(f'L {x+overall_w/2.0} {y}')
    elif alter2s_to_bounds is None:
        ah = alter1s_to_bounds[3]-alter1s_to_bounds[1]-2*_amt_
        d_array.append(f'L {x-overall_w/2.0} {y+ah+  _amt_}  C {x-overall_w/2.0} {y+ah+2*_amt_} {x-overall_w/2.0} {y+ah+2*_amt_} {x-narrow_w/2.0}  {y+ah+2*_amt_}')
        d_array.append(f'L {x+narrow_w/2.0}  {y+ah+2*_amt_}  C {x+overall_w/2.0} {y+ah+2*_amt_} {x+overall_w/2.0} {y+ah+2*_amt_} {x+overall_w/2.0} {y+ah+  _amt_}')
        d_array.append(f'L {x+overall_w/2.0} {y}')
    else:
        ah  = alter1s_to_bounds[3]-alter1s_to_bounds[1]-2*_amt_
        d_array.append(f'L {x-overall_w/2.0} {y+ah+  _amt_}  C {x-overall_w/2.0} {y+ah+2*_amt_} {x-overall_w/2.0} {y+ah+2*_amt_} {x-narrow_w/2.0}  {y+ah+2*_amt_}')
        a2y  = alter2s_to_bounds[1] + 2*r_pref
        d_array.append(f'L {x-narrow_w/2.0}  {a2y}           C {x-overall_w/2.0} {a2y}          {x-overall_w/2.0} {a2y}          {x-overall_w/2.0} {a2y+2*_amt_}')
        a2y2 = alter2s_to_bounds[3] - 2*_amt_
        d_array.append(f'L {x-overall_w/2.0} {a2y2}')
        d_array.append(f'C {x-overall_w/2.0} {a2y2+2*_amt_} {x-overall_w/2.0} {a2y2+2*_amt_} {x-narrow_w/2.0}  {a2y2+2*_amt_}')
        d_array.append(f'L {x+narrow_w/2.0}  {a2y2+2*_amt_}  C {x+overall_w/2.0} {a2y2+2*_amt_} {x+overall_w/2.0} {a2y2+2*_amt_} {x+overall_w/2.0} {a2y2}')
        d_array.append(f'L {x+overall_w/2.0} {a2y +2*_amt_}  C {x+overall_w/2.0} {a2y}          {x+overall_w/2.0} {a2y}          {x+narrow_w/2.0}  {a2y}')
        d_array.append(f'L {x+narrow_w/2.0}  {y+ah+2*_amt_}  C {x+overall_w/2.0} {y+ah+2*_amt_} {x+overall_w/2.0} {y+ah+2*_amt_} {x+overall_w/2.0} {y+ah+  _amt_}')
        d_array.append(f'L {x+overall_w/2.0} {y}')

    if alter1s_fm_bounds is None:
        d_array.append(f'L {x+overall_w/2.0} {y-  _amt_}  C {x+overall_w/2.0} {y-2*_amt_} {x+overall_w/2.0} {y-2*_amt_} {x+narrow_w/2.0}  {y-2*_amt_}')
        d_array.append(f'L {x-narrow_w/2.0}  {y-2*_amt_}  C {x-overall_w/2.0} {y-2*_amt_} {x-overall_w/2.0} {y-2*_amt_} {x-overall_w/2.0} {y-  _amt_}')
        d_array.append(f'L {x-overall_w/2.0} {y}')
    elif alter2s_fm_bounds is None:
        ah = alter1s_fm_bounds[3]-alter1s_fm_bounds[1]-2*_amt_
        d_array.append(f'L {x+overall_w/2.0} {y-ah-  _amt_}  C {x+overall_w/2.0} {y-ah-2*_amt_} {x+overall_w/2.0} {y-ah-2*_amt_} {x+narrow_w/2.0}  {y-ah-2*_amt_}')
        d_array.append(f'L {x-narrow_w/2.0}  {y-ah-2*_amt_}  C {x-overall_w/2.0} {y-ah-2*_amt_} {x-overall_w/2.0} {y-ah-2*_amt_} {x-overall_w/2.0} {y-ah-  _amt_}')
        d_array.append(f'L {x-overall_w/2.0} {y}')
    else:
        ah  = alter1s_fm_bounds[3]-alter1s_fm_bounds[1]-2*_amt_
        d_array.append(f'L {x+overall_w/2.0} {y-ah-  _amt_}  C {x+overall_w/2.0} {y-ah-2*_amt_} {x+overall_w/2.0} {y-ah-2*_amt_} {x+narrow_w/2.0}  {y-ah-2*_amt_}')
        a2y  = alter2s_fm_bounds[3] - 2*r_pref
        d_array.append(f'L {x+narrow_w/2.0}  {a2y}           C {x+overall_w/2.0} {a2y}          {x+overall_w/2.0} {a2y}          {x+overall_w/2.0} {a2y-2*_amt_}')
        a2y2 = alter2s_fm_bounds[1] + 2*_amt_
        d_array.append(f'L {x+overall_w/2.0} {a2y2}')
        d_array.append(f'C {x+overall_w/2.0} {a2y2-2*_amt_} {x+overall_w/2.0} {a2y2-2*_amt_} {x+narrow_w/2.0}  {a2y2-2*_amt_}')
        d_array.append(f'L {x-narrow_w/2.0}  {a2y2-2*_amt_}  C {x-overall_w/2.0} {a2y2-2*_amt_} {x-overall_w/2.0} {a2y2-2*_amt_} {x-overall_w/2.0} {a2y2}')
        d_array.append(f'L {x-overall_w/2.0} {a2y -2*_amt_}  C {x-overall_w/2.0} {a2y}          {x-overall_w/2.0} {a2y}          {x-narrow_w/2.0}  {a2y}')
        d_array.append(f'L {x-narrow_w/2.0}  {y-ah-2*_amt_}  C {x-overall_w/2.0} {y-ah-2*_amt_} {x-overall_w/2.0} {y-ah-2*_amt_} {x-overall_w/2.0} {y-ah-  _amt_}')
        d_array.append(f'L {x-overall_w/2.0} {y}')

    svg.append(f'<path d="{"".join(d_array)}" stroke="{self.rt_self.co_mgr.getTVColor("axis","major")}" stroke-width="2.0" fill="none" />')

    return ''.join(svg), (x-max_w/2, y-max_h/2, x+max_w/2, y+max_h/2)

sl.renderBin = renderBin.__get__(sl, SpreadLines)

alter_inter_d = 192
max_bin_w     = 64
max_bin_h     = 450*2
svg = []
svg.append(f'<svg x="0" y="0" width="{1536}" height="{768}" viewBox="0 0 2048 1024">')
svg.append(f'<rect x="0" y="0" width="{2048}" height="{1024}" fill="{rt.co_mgr.getTVColor("background","default")}" />')
_bins_ordered_ = list(sl.bin_to_timestamps.keys())
_bins_ordered_.sort()
x = max_bin_w
for _bin_ in _bins_ordered_:
    _svg_, _bounds_ = sl.renderBin(_bin_, x, (1024-max_bin_h)/2 + max_bin_h/2, max_bin_w, max_bin_h)
    svg.append(_svg_)
    xmin, ymin, xmax, ymax = _bounds_
    x += alter_inter_d + max_bin_w
svg.append('</svg>')
rt.tile([''.join(svg)])